Note that the given notebook is the shorten version of the original one, which is around 7-9 times longer, but this version simply does not include visualisation, and analysis insights, simpy step by step comments. The premises for this notebook, was not to train the model to "fit" the synthetically (but still does good in this case) created data, but to truly properly generalize it to the real world data. Thus the columns like "Name" are not a good way of expressing the real world data, since even tho that synthetically created data, could fit it, it won't truly generalize to the real data, thus the notebook was created for aforementioned premise.

1. The data is downloaded.
2. Everything needed is imported, and the data is transformed in dataframes.
3. The unneccessary columns are dropped, they are in case of City and Name both too dimensional and irrelevant, additionally the profession amount of different professions, and their unrealistic data (e.g. doctors have one of the lowest amounts of depression which is absolutely untrue).
4. The health habits are inferred for data via assuming the mean ( note that this data will later be in ordinal form, properly adjusted for assumption: Depression -> 1, Healthy -> 0), i.e. the 'moderate' for each Nan cell, the number of nans are extremely low, and this will be also true for the next two columns. The next two columns: Degree is filled with 'Class 12' the mode, but the underlying assumption is the higher the education level the more likely the person is to properly write the data in the cell (the smarter on average the person is, and more collected... on average), and the thrid column is: filled with 5.5(implicitly) because it is slightly lower than average (people with depression are more likely to be not willing or write something else in such fields), but average won't be achieveable due to unneccessary PCA reverse, without any proper reason.
5. Then the data is split to avoid it's leakage, for the transformations are about to begin.
6. The cgpa is reasonably splitted to assure proper division within the names of tha values it will be OHE'ed because the Nans are here not because of the wrong data, but they are simply caused by people who do not attend the university, thus it will be OHE'ed as 'N'. And the CGPA will be replaced by cgpa_grouped.
7. The PCA (i.e. dimensionality reduction) is then adjusted on sleep duration column, for we need it for PCA, and to make it easier for the model to learn that this is sort of ordinal relation.
8. Then the columns: ['Academic Pressure', 'Work Pressure', 'Study Satisfaction', 'Job Satisfaction'], are then typed for strings, because we will need to name the nans as 'N', 'N' because they occurred in a true pattern, and they are intertwined with each other, representing students and works (in details this is explained (as everything else) in the longer version of this notebook), thus we cannot leave it as numerical values, we must encode them and as nans assume 'N'. This is very important to not infer them, they MUST not exist.
9. Then the depression is mapped accordingly with convention, i.e. illness -> 1 and healthy -> 0.
10. After that, the degree is swapped for it's depression targeted version. There are too much degrees, for general amount of rows (additionally the nans are filled with the mean) thus through dimensionality reduction, and additionally the degree column, should to some degree influence whether the given person has this medical condition.
11. Then the columns: [
    'Have you ever had suicidal thoughts ?',
    'Working Professional or Student',
    'Family History of Mental Illness',
    'Gender',
    'Academic Pressure',
    'Work Pressure',
    'Study Satisfaction',
    'Job Satisfaction',
    'cgpa_grouped'
] are OHE'ed, most of them I have previously mentioned the reasons for them to be OHE'ed, [
    'Have you ever had suicidal thoughts ?',
    'Working Professional or Student',
    'Family History of Mental Illness',
    'Gender' ] All of them contains a non-ordinal binary classes, thus the OHE is the best choice, since we do not have to worry, in this case about dimensionality increase, to a significant extent.]
12. Then column: 'Dietary habits' is ordinally encoded, as the sole column preprocessed with this fashion in the dataset, it's values are truly aligned in the ordinal form.
13. The columns: ['Age', 'Dietary Habits', 'Work/Study Hours', 'Financial Stress', 'Degree_targeted', 'sd_grouped'] are standardized, I thought about min-max normalization, but I changed my view on this idea, mainly due to the fact that I did not want to use DL, and for ML this is in most cases a prefered way, the min-max patterns will be harder to infer for ML models, that I am willing to assess i.e. fit to this data.
14. After the feature engineering, data analysis and preprocessing is over, I splitted the data into X and y.
15. Firstly I have used an XGBoost ensemble model, for the number of dimensions isn't that high, and additionally, random forest, might underfit the data, thus I decided to use XGBoost, then after using it with Optuna, I decided to use SVM, in order to fit this small-medium sized dataset to fit the dimensionality of the data into SVM, thus I selected exactly these kernels, the poly, line would not fit the data properly. I have used Optuna to find the best parameters within the given bounds for a given model, and then with these parameters I have trained a model on new data, i.e. combined train, val and test data.
16. Finally achieveing good and sufficient accuracy, the model was fitted with entire data, on best parameters, in order to make predictions on Kaggle dataset.
17. At the end, after saving the predictions to the submission.csv file, I have saved the model via joblib.

In [1]:
!pip install --upgrade scikit-learn
!pip install optuna
!wget -O dataset.csv "https://raw.githubusercontent.com/LozinskiMatthew/Depression-Predicting-Model/refs/heads/main/final_depression_dataset_1.csv"
!wget -O test.csv "https://raw.githubusercontent.com/LozinskiMatthew/Depression-Predicting-Model/refs/heads/main/test.csv"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.8/231.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 5.3 MB/s eta 0:00:00
--2025-03-11 10:21:08--  https://raw.githubusercontent.com/LozinskiMatthew/Depression-Predicting-Model/refs/heads/main/final_depression_dataset_1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 259561 (253K) [text/plain]
Saving to: ‘dataset.csv’

dataset.csv         100%[===================>] 253.48K  --.-KB/s    in 0.04s   

2025-03-11 10:21:08 (6.65 MB/s) - ‘dataset.csv’ saved [259561/259561]

--2025-03-11 10:21:08--  https://raw.githubusercontent.com/LozinskiMatthew/Depression-Predicting-Model/refs/heads/main/test.cs

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import optuna
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
import joblib # It shall handle this better than pickle

data = pd.read_csv("/content/dataset.csv")
final_dataset = pd.read_csv("/content/test.csv")
data.columns

Index(['Name', 'Gender', 'Age', 'City', 'Working Professional or Student',
       'Profession', 'Academic Pressure', 'Work Pressure', 'CGPA',
       'Study Satisfaction', 'Job Satisfaction', 'Sleep Duration',
       'Dietary Habits', 'Degree', 'Have you ever had suicidal thoughts ?',
       'Work/Study Hours', 'Financial Stress',
       'Family History of Mental Illness', 'Depression'],
      dtype='object')

In [3]:
final_dataset.drop(columns=['City', 'Name', 'Profession'], axis=1, inplace=True)
data.drop(columns=['City', 'Name', 'Profession'], axis=1, inplace=True)

In [4]:
health_habits = ('Moderate', 'Unhealthy', 'Healthy')
final_dataset.loc[~final_dataset['Dietary Habits'].isin(health_habits), 'Dietary Habits'] = 'Moderate'
print(final_dataset['Dietary Habits'].value_counts())

train_degrees = set(data['Degree'].unique())
final_dataset.loc[~final_dataset['Degree'].isin(train_degrees), 'Degree'] = 'Class 12'
print(final_dataset['Degree'].value_counts())

train_sd = set(data['Sleep Duration'].unique())
final_dataset.loc[~final_dataset['Sleep Duration'].isin(train_sd), 'Sleep Duration'] = '5-6 hours'
print(final_dataset['Sleep Duration'].value_counts())

Dietary Habits
Moderate     33048
Unhealthy    30786
Healthy      29966
Name: count, dtype: int64
Degree
Class 12    9898
B.Ed        7762
B.Arch      6037
B.Com       5439
B.Pharm     3987
BCA         3869
M.Ed        3707
MCA         3438
BBA         3387
BSc         3314
LLM         3133
MSc         3096
M.Tech      3017
M.Pharm     2995
LLB         2938
B.Tech      2885
BHM         2776
BA          2490
MBA         2470
ME          2391
MD          2289
BE          2166
MHM         2149
M.Com       2128
PhD         2073
MBBS        2033
MA          1933
Name: count, dtype: int64
Sleep Duration
Less than 5 hours    25661
7-8 hours            24491
More than 8 hours    22190
5-6 hours            21458
Name: count, dtype: int64


In [5]:
train_data, temp_data = train_test_split(data, test_size=0.2, shuffle=True, random_state=42)

In [6]:
#The cgpa_grouped should be instead on CGPA.
def categorize_by_cgpa(cgpa):
  if np.isnan(cgpa):
    return 'N'
  elif cgpa < 6 and cgpa >= 5:
    return '5-6'
  elif cgpa < 7 and cgpa >= 6:
    return '6-7'
  elif cgpa < 8 and cgpa >= 7:
    return '7-8'
  elif cgpa < 9 and cgpa >= 8:
    return '8-9'
  elif cgpa <= 10 and cgpa >= 9:
    return '9-10'

train_data['cgpa_grouped'] = train_data['CGPA'].apply(categorize_by_cgpa)
temp_data['cgpa_grouped'] = temp_data['CGPA'].apply(categorize_by_cgpa)
final_dataset['cgpa_grouped'] = final_dataset['CGPA'].apply(categorize_by_cgpa)

In [7]:
def adjust_sleep_duration(sd):
  if '5' in sd and '6' in sd:
    return 5.5
  if '5' in sd and '6' not in sd:
    return 4.5
  if '7' in sd and '8' in sd:
    return 7.5
  if '8' in sd and '7' not in sd:
    return 9.0

train_data['sd_grouped'] = train_data['Sleep Duration'].apply(adjust_sleep_duration)
temp_data['sd_grouped'] = temp_data['Sleep Duration'].apply(adjust_sleep_duration)
final_dataset['sd_grouped'] = final_dataset['Sleep Duration'].apply(adjust_sleep_duration)

In [8]:
train_data.drop(columns=['CGPA', 'Sleep Duration'], axis = 1, inplace = True)
temp_data.drop(columns=['CGPA', 'Sleep Duration'], axis = 1, inplace = True)
final_dataset.drop(columns=['CGPA', 'Sleep Duration'], axis = 1, inplace = True)

In [9]:
columns_to_convert = ['Academic Pressure', 'Work Pressure', 'Study Satisfaction', 'Job Satisfaction']
dsets = (train_data, temp_data, final_dataset)

for df in dsets:
    for col in columns_to_convert:
        df[col] = df[col].astype(str)

In [10]:
cols = ('Academic Pressure', 'Work Pressure', 'Study Satisfaction', 'Job Satisfaction')

for col in cols:
  for ds in dsets:
    ds[col] = ds[col].fillna('N')

In [11]:
train_data['Degree'].value_counts()

,count
Degree,
Class 12,220
B.Ed,93
MCA,89
B.Com,85
BHM,81
B.Arch,79
BCA,79
MSc,78
B.Pharm,73


In [12]:
train_data['Depression'] = train_data['Depression'].map({'No': 0, 'Yes': 1})
temp_data['Depression'] = temp_data['Depression'].map({'No': 0, 'Yes': 1})

In [13]:
global_mean = train_data['Depression'].mean()

train_target_encoding = train_data.groupby('Degree')['Depression'].mean()

train_data['Degree_targeted'] = train_data['Degree'].map(train_target_encoding).fillna(global_mean)
temp_data['Degree_targeted'] = temp_data['Degree'].map(train_target_encoding).fillna(global_mean)
final_dataset['Degree_targeted'] = final_dataset['Degree'].map(train_target_encoding).fillna(global_mean)


In [14]:
cat_cols = [
    'Have you ever had suicidal thoughts ?',
    'Working Professional or Student',
    'Family History of Mental Illness',
    'Gender',
    'Academic Pressure',
    'Work Pressure',
    'Study Satisfaction',
    'Job Satisfaction',
    'cgpa_grouped'
]

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoder.fit(train_data[cat_cols])

train_encoded = encoder.transform(train_data[cat_cols])
temp_encoded = encoder.transform(temp_data[cat_cols])
final_encoded = encoder.transform(final_dataset[cat_cols])

encoded_cols = encoder.get_feature_names_out(cat_cols)

train_encoded_df = pd.DataFrame(train_encoded, columns=encoded_cols, index=train_data.index)
temp_encoded_df = pd.DataFrame(temp_encoded, columns=encoded_cols, index=temp_data.index)
final_encoded_df = pd.DataFrame(final_encoded, columns=encoded_cols, index=final_dataset.index)

train_data = train_data.drop(columns=cat_cols)
temp_data = temp_data.drop(columns=cat_cols)
final_dataset = final_dataset.drop(columns=cat_cols)

train_data = train_data.join(train_encoded_df)
temp_data = temp_data.join(temp_encoded_df)
final_dataset = final_dataset.join(final_encoded_df)

In [15]:
train_data.shape

(2044, 46)

In [16]:
final_dataset.shape

(93800, 46)

In [17]:
temp_data.shape

(512, 46)

In [18]:
train_data.drop('Degree', axis=1, inplace=True)
temp_data.drop('Degree', axis=1, inplace=True)
final_dataset.drop('Degree', axis=1, inplace=True)

In [19]:
ord = OrdinalEncoder(categories=[['Healthy', 'Moderate', 'Unhealthy']])
train_data['Dietary Habits'] = ord.fit_transform(train_data[['Dietary Habits']])
temp_data['Dietary Habits'] = ord.transform(temp_data[['Dietary Habits']])
final_dataset['Dietary Habits'] = ord.transform(final_dataset[['Dietary Habits']])

In [20]:
cols_to_scale = ['Age', 'Dietary Habits', 'Work/Study Hours', 'Financial Stress', 'Degree_targeted', 'sd_grouped']

stscaler = StandardScaler()
train_scaled = stscaler.fit_transform(train_data[cols_to_scale])
temp_scaled = stscaler.transform(temp_data[cols_to_scale])
final_scaled = stscaler.transform(final_dataset[cols_to_scale])

train_scaled = pd.DataFrame(train_scaled, columns=cols_to_scale, index=train_data.index)
temp_scaled = pd.DataFrame(temp_scaled, columns=cols_to_scale, index=temp_data.index)
final_scaled = pd.DataFrame(final_scaled, columns=cols_to_scale, index=final_dataset.index)

train_data = pd.concat([train_data.drop(columns=cols_to_scale), train_scaled], axis=1)
temp_data = pd.concat([temp_data.drop(columns=cols_to_scale), temp_scaled], axis=1)
final_dataset = pd.concat([final_dataset.drop(columns=cols_to_scale), final_scaled], axis=1)

In [21]:
validation_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

In [22]:
train_data.isna().sum()

,0
Depression,0
Have you ever had suicidal thoughts ?_No,0
Have you ever had suicidal thoughts ?_Yes,0
Working Professional or Student_Student,0
Working Professional or Student_Working Professional,0
Family History of Mental Illness_No,0
Family History of Mental Illness_Yes,0
Gender_Female,0
Gender_Male,0
Academic Pressure_1.0,0


In [23]:
# train_data.drop(columns=['Degree_targeted'], axis=1, inplace=True)

In [24]:
# validation_data.drop(columns=['Degree_targeted'], axis=1, inplace=True)

In [25]:
X_train = train_data.drop(columns=['Depression'])
y_train = train_data['Depression']
X_val = validation_data.drop(columns=['Depression'])
y_val = validation_data['Depression']
X_test = test_data.drop(columns=['Depression'])
y_test = test_data['Depression']

In [26]:
X_combined = pd.concat([X_train, X_val, X_test], ignore_index=True)
y_combined = pd.concat([y_train, y_val, y_test], ignore_index=True)

In [27]:
diff_columns = set(final_dataset.columns) - set(train_data.columns)
print("Columns in final_dataset but not in train_data:", diff_columns)

Columns in final_dataset but not in train_data: {'id'}


In [28]:
"""
def objective(trial):
    param = {
        'booster': 'gbtree',
        'n_estimators': trial.suggest_int('n_estimators', 20, 80),
        'max_depth': trial.suggest_int('max_depth', 3, 5),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.15, 0.4),
        'subsample': trial.suggest_uniform('subsample', 0.8, 1.0),
        'gamma': trial.suggest_loguniform('gamma', 1e-4, 1e-2),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),
        'objective': 'binary:logistic',
        'eval_metric': 'mae',
        'verbosity': 1,
        'use_label_encoder': False,
        'random_state': 42,
    }

    model = XGBClassifier(**param)

    score = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=4).mean()

    return score

study = optuna.create_study(direction='maximize')

study.optimize(objective, n_trials=80)

print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")
"""

'\ndef objective(trial):\n    param = {\n        \'booster\': \'gbtree\',\n        \'n_estimators\': trial.suggest_int(\'n_estimators\', 20, 80),\n        \'max_depth\': trial.suggest_int(\'max_depth\', 3, 5),\n        \'learning_rate\': trial.suggest_loguniform(\'learning_rate\', 0.15, 0.4),\n        \'subsample\': trial.suggest_uniform(\'subsample\', 0.8, 1.0),\n        \'gamma\': trial.suggest_loguniform(\'gamma\', 1e-4, 1e-2),\n        \'reg_alpha\': trial.suggest_loguniform(\'reg_alpha\', 1e-8, 1.0),\n        \'objective\': \'binary:logistic\',\n        \'eval_metric\': \'mae\',\n        \'verbosity\': 1,\n        \'use_label_encoder\': False,\n        \'random_state\': 42,\n    }\n\n    model = XGBClassifier(**param)\n\n    score = cross_val_score(model, X_train, y_train, scoring=\'accuracy\', cv=4).mean()\n\n    return score\n\nstudy = optuna.create_study(direction=\'maximize\')\n\nstudy.optimize(objective, n_trials=80)\n\nprint("Best trial:")\ntrial = study.best_trial\nprint(f"

In [29]:
def objective(trial):
    param = {
        'C': trial.suggest_loguniform('C', 0.6, 0.95),
        'kernel': trial.suggest_categorical('kernel', ['rbf', 'sigmoid']),
        'gamma': trial.suggest_categorical('gamma', ['scale', 'auto']),
        'probability': True
    }

    model = SVC(**param, random_state=42)

    score = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=4).mean()

    return score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=80)

print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-03-11 10:21:17,381] A new study created in memory with name: no-name-3be5332c-2617-419c-9c3b-a78d041d3bec
<ipython-input-29-bd03f84f3c66>:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'C': trial.suggest_loguniform('C', 0.6, 0.95),
[I 2025-03-11 10:21:19,868] Trial 0 finished with value: 0.9432485322896281 and parameters: {'C': 0.6299201324240656, 'kernel': 'rbf', 'gamma': 'scale'}. Best is trial 0 with value: 0.9432485322896281.
<ipython-input-29-bd03f84f3c66>:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'C': trial.suggest_loguniform('C', 0.6, 0.95),
[I 2025-03-11 10:21:21,896] Trial 1 finished with value: 0.949119373776908 and parameters: {'C': 0.8685022

Best trial:
  Value: 0.9510763209393346
  Params:
    C: 0.9325533851317857
    kernel: rbf
    gamma: auto


In [30]:
best_params = study.best_trial.params

final_model = SVC(**best_params)
final_model.fit(X_combined, y_combined)

SVC(C=0.9325533851317857, gamma='auto')

In [31]:
"""
predictions = final_model.predict(X_val)

acc_score = accuracy_score(y_pred=predictions, y_true=y_val)
conf_matrix = confusion_matrix(y_pred=predictions, y_true=y_val)
print(acc_score)
print(conf_matrix)
"""

'\npredictions = final_model.predict(X_val)\n\nacc_score = accuracy_score(y_pred=predictions, y_true=y_val)\nconf_matrix = confusion_matrix(y_pred=predictions, y_true=y_val)\nprint(acc_score)\nprint(conf_matrix)\n'

In [32]:
ids = final_dataset['id']

test_features = final_dataset.drop('id', axis=1)

predictions = final_model.predict(test_features)

submission = pd.DataFrame({
    'id': ids,
    'target': predictions
})

submission.to_csv('submission.csv', index=False)


In [33]:
joblib.dump(final_model, 'best_model.pkl')

['best_model.pkl']